<a href="https://colab.research.google.com/github/saiashirwad/relation-prediction-2/blob/master/Relation_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd /content/drive/My Drive/Thesis/code/relation-prediction-2

/content/drive/My Drive/Thesis/code/relation-prediction-2


In [0]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import matplotlib.pyplot as plt


In [0]:
from layers import *

In [0]:
pip install torchkge --quiet

In [0]:
pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html --quiet

In [0]:
from tqdm import * 
import torch 
import torch.nn as nn 
from functools import reduce 
from operator import mul 
from torch_scatter import scatter 
from sklearn.metrics.pairwise import pairwise_distances 

In [0]:
from train import *

In [0]:
kg_train, kg_test, kg_val = load_fb15k237()

In [0]:
args = Args(100, 200, 100, 2, 100, 80000, 0.001, 10, 'cuda', 'sgd')

In [0]:
n_ent, n_rel = kg_train.n_ent, kg_train.n_rel
total_triplets = get_valid_triplets(kg_train, kg_test, kg_val)

Number of unique triplets: 620232


In [0]:
def loss_func2(triplets, neg_sampling_ratio, ent_embed, rel_embed, device='cpu'):
    """
    Triplets order: src, dst, rel
    """
    n = len(triplets)
    if type(triplets) == np.ndarray:
        triplets = torch.from_numpy(triplets)

    pos_triplets = triplets[:n // (neg_sampling_ratio + 1)]
    pos_triplets = torch.cat([pos_triplets for _ in range(neg_sampling_ratio)])

    neg_triplets = triplets[n // (neg_sampling_ratio + 1):]


    src_embed_ = ent_embed[pos_triplets[:, 0]]
    dst_embed_ = ent_embed[pos_triplets[:, 1]]
    rel_embed_ = rel_embed[pos_triplets[:, 2]]

    x = src_embed_ + rel_embed_ - dst_embed_
    pos_norm = torch.norm(x, p=2, dim=1)


    src_embed_ = ent_embed[neg_triplets[:, 0]]
    dst_embed_ = ent_embed[neg_triplets[:, 1]]
    rel_embed_ = rel_embed[neg_triplets[:, 2]]

    x = src_embed_ + rel_embed_ - dst_embed_
    neg_norm = torch.norm(x, p=2, dim=1)

    y = torch.ones(len(pos_triplets)).to(device)

    loss_fn = nn.MarginRankingLoss(margin=5)
    loss = loss_fn(pos_norm, neg_norm, y)

    return loss


In [0]:
dataloader = DataLoader(kg_train, batch_size=args.batch_size, shuffle=False, pin_memory=cuda.is_available())
ent_embed, rel_embed = get_init_embed()

In [0]:
class SNAFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, edge, edge_w, N, E, out_features):
        a = torch.sparse_coo_tensor(
            edge, edge_w, torch.Size([N, N, out_features]))
        b = torch.sparse.sum(a, dim=1)
        ctx.N = b.shape[0]
        ctx.outfeat = b.shape[1]
        ctx.E = E
        ctx.indices = a._indices()[0, :]

        return b.to_dense()

    @staticmethod
    def backward(ctx, grad_output):
        grad_values = None
        if ctx.needs_input_grad[1]:
            edge_sources = ctx.indices

            if(torch.cuda.is_available()):
                edge_sources = edge_sources.cuda()

            grad_values = grad_output[edge_sources]
        return None, grad_values, None, None, None

class SparseNeighborhoodAggregation(nn.Module):
    def forward(self, edge, edge_w, N, E, out_features):
        return SNAFunction.apply(edge, edge_w, N, E, out_features)


![](https://i.imgur.com/2v9bxG1.png)

In [0]:
class KGLayer(nn.Module):
    def __init__(self, n_entities, n_relations, ent_embed, rel_embed, in_dim, out_dim, concat=True, device="cuda"):
        super(KGLayer, self).__init__()

        self.n_entities = n_entities
        self.n_relations = n_relations
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.device = device

        self.a = nn.Linear(3 * in_dim, out_dim).to(device)
        nn.init.xavier_normal_(self.a.weight.data, gain=1.414)

        self.concat = concat

        self.a_2 = nn.Linear(out_dim, 1).to(device)
        nn.init.xavier_normal_(self.a_2.weight.data, gain=1.414)

        self.sparse_neighborhood_aggregation = SparseNeighborhoodAggregation()

        self.ent_embed = nn.Embedding(n_entities, in_dim).to(device)
        self.rel_embed = nn.Embedding(n_relations, in_dim).to(device)
        
        # nn.init.xavier_normal_(self.ent_embed.weight.data, 1.414)
        # nn.init.xavier_normal_(self.rel_embed.weight.data, 1.414)
        
        self.ent_embed.weight = nn.Parameter(ent_embed.to("cuda"))
        self.rel_embed.weight = nn.Parameter(rel_embed.to("cuda"))
    
    def forward(self, triplets, eval=False):

        N = self.n_entities

        h = torch.cat((
            self.ent_embed(triplets[:, 0]),
            self.ent_embed(triplets[:, 1]),
            self.rel_embed(triplets[:, 2])
        ), dim=1)
        c = self.a(h)
        b = -F.leaky_relu(self.a_2(c))
        e_b = torch.exp(b)

        temp = triplets.t()
        edges = torch.stack([temp[0], temp[1]])

        ebs = self.sparse_neighborhood_aggregation(edges, e_b, N, e_b.shape[0], 1)
        temp1 = e_b * c

        hs = self.sparse_neighborhood_aggregation(edges, temp1,  N, e_b.shape[0], self.out_dim)

        ebs[ebs == 0] = 1e-12


        h_ent = hs / ebs

        index = triplets[:, 2]
        h_rel = scatter(temp1, index=index, dim=0, reduce="mean")
        
        if self.concat:
            return F.elu(h_ent), F.elu(h_rel)
        else:
            return h_ent, h_rel

In [0]:
model = KGLayer(n_ent, n_rel, ent_embed, rel_embed, 100, 100, True, "cuda")

In [0]:
optimizer = SGD(model.parameters(), lr=0.001)

In [0]:
batches = [b for b in dataloader]

In [0]:
for epoch in tnrange(5):
    losses = []
    for i in range(len(batches)):
        batch = batches[i]
        triplets = torch.stack(batch)
        triplets, labels, nodes, edges = negative_sampling(triplets, n_ent, args.negative_rate)
        triplets, labels = triplets.to(args.device), labels.to(args.device)
    
        model.zero_grad()
    
        # start = time.time()
        model.train()
        ent_embed_, rel_embed_ = model(triplets)
        loss = loss_func2(triplets, args.negative_rate, ent_embed_, rel_embed_, device="cuda")
        loss.backward()
        optimizer.step()
    
        # del triplets
        # del labels
        # del ent_embed_
        # del rel_embed_
        # del nodes 
        # del edges  
        torch.cuda.empty_cache()
    
        losses.append(loss.item())
        # print(loss.item())
        # del loss 
        torch.cuda.empty_cache()
    
    # print(f"epoch: {epoch}, loss: {sum(losses) / len(losses)}")
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """Entry point for launching an IPython kernel.


In [0]:
torch.save(model.state_dict(), "model.save")

In [0]:
model.load_state_dict(torch.load("model.save"))

<All keys matched successfully>

In [0]:
dl = DataLoader(kg_val, 10000, shuffle=True)
data = [d for d in dl]
triplets = data[0]

In [0]:
# eval 
triplets = torch.stack(data[0])
triplets, labels, nodes, edges = negative_sampling(triplets, n_ent, 0)
triplets, labels = triplets.to(args.device), labels.to(args.device)
model.eval() 

ee, re = model(triplets)

In [0]:
ee = model.ent_embed.weight.cpu().detach()
re = model.rel_embed.weight.cpu().detach() 

In [0]:
e = ee.cpu().detach()

In [0]:
r = re.cpu().detach()

In [0]:
src_ = [s.item() for s in triplets[:, 0]]
dst_ = [d.item() for d in triplets[:, 1]]
rel_ = [r.item() for r in triplets[:, 2]]

In [0]:
src = e[src_]
dst = e[dst_]
rel = r[rel_]

In [0]:
dist = pairwise_distances(dst - rel, e, metric="euclidean") 

In [0]:
rankArrayHead = np.argsort(dist, axis=1)

In [0]:
rankListHead =  [int(np.argwhere(e[1] == e[0])) for e in zip(src_, rankArrayHead)]

In [0]:
isHit10Head = [x for x in rankListHead if x < 10]

In [0]:
sum(rankListHead) / len(rankListHead)

7254.7366

In [0]:
len(isHit10Head)

49

In [0]:
dist2 = pairwise_distances(src + rel, e, metric="euclidean")

In [0]:
rankArrayTail = np.argsort(dist2, axis=1)

In [0]:
rankListTail = [int(np.argwhere(e[1] == e[0])) for e in zip(dst_, rankArrayTail)]

In [0]:
isHit10Tail = [x for x in rankListTail if x < 10]

In [0]:
len(isHit10Tail)

7

In [0]:
sum(rankListTail) / len(rankListTail)

7202.4886

In [0]:
!nvidia-smi

Thu Apr 16 09:38:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    71W / 149W |    661MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  